In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.0 MB/s eta 0:00:00


In [ ]:
pip install focal-loss

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import transformers

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/spam_consolidated.xlsx")

In [ ]:
df.shape

(264980, 2)

In [ ]:
df = df.dropna()

In [ ]:
df.shape

(264969, 2)

In [ ]:
df

,text,label
0,"There are, for instance, ""gay"" transsexuals...",NSFW
1,"There are, for instance, ""gay"" transsexuals...",NSFW
2,- Angels or demons has intercourse with mortals,NSFW
3,"fuckbitch sites, the dating sites, cheating...",NSFW
4,"fuckpuppet sites, the dating sites, cheatin...",NSFW
...,...,...
264975,if ur still on twitter tell carlton i said his...,NSFW
264976,when i first got on here and said i hate trump...,NSFW
264977,was macht der moslem wenn der zion gegen seine...,NSFW
264978,it is awful look at world demographics asians ...,NSFW


In [ ]:
df["label"].value_counts()

SFW     147536
NSFW    117433
Name: label, dtype: int64

In [ ]:
df1 = df[df["label"]=="SFW"]

In [ ]:
df1

,text,label
70371,Explanation\nWhy the edits made under my usern...,SFW
70372,D'aww! He matches this background colour I'm s...,SFW
70373,"Hey man, I'm really not trying to edit war. It...",SFW
70374,"""\nMore\nI can't make any real suggestions on ...",SFW
70375,"You, sir, are my hero. Any chance you remember...",SFW
...,...,...
242858,OnPassive is not a swindle,SFW
242859,OnPassive is a company that offers AI-powered ...,SFW
242860,It is important to note that OnPassive is a le...,SFW
242861,There have been no concrete evidence or report...,SFW


In [ ]:
df_final = pd.concat([df1[:100000],df[df["label"]=="NSFW"]], axis=0)

In [ ]:
dic = {"NSFW":1, "SFW":0}
df_final["label"] = df_final["label"].map(dic)

In [ ]:
df_final

,text,label
70371,Explanation\nWhy the edits made under my usern...,0
70372,D'aww! He matches this background colour I'm s...,0
70373,"Hey man, I'm really not trying to edit war. It...",0
70374,"""\nMore\nI can't make any real suggestions on ...",0
70375,"You, sir, are my hero. Any chance you remember...",0
...,...,...
264975,if ur still on twitter tell carlton i said his...,1
264976,when i first got on here and said i hate trump...,1
264977,was macht der moslem wenn der zion gegen seine...,1
264978,it is awful look at world demographics asians ...,1


In [ ]:
df_final["label"].value_counts()


1    117433
0    100000
Name: label, dtype: int64

In [ ]:
df_final = df_final.dropna()

In [ ]:
df_final.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
from transformers import DistilBertTokenizerFast

# Instantiate DistilBERT tokenizer...we use the Fast version to optimize runtime
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)
MAX_LENGTH = 128


# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):

    input_ids = []
    attention_mask = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='longest', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])


    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)




In [ ]:
X= df_final['text']
y=df_final['label']

# using the train test split function
X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

In [ ]:
type(X_train)

pandas.core.series.Series

In [ ]:
X_train = X_train.astype(str)
X_test = X_test.astype(str)


In [ ]:
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

# # Encode X_valid
# X_valid_ids, X_valid_attention = batch_encode(tokenizer, X_valid.tolist())

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.tolist())

In [ ]:
X_train_ids.shape

TensorShape([163074, 128])

In [ ]:
X_train_attention.shape

TensorShape([163074, 128])

In [ ]:
from transformers import TFDistilBertModel, DistilBertConfig

DISTILBERT_DROPOUT = 0.2
DISTILBERT_ATT_DROPOUT = 0.2

# Configure DistilBERT's initialization
config = DistilBertConfig(dropout=DISTILBERT_DROPOUT,
                          attention_dropout=DISTILBERT_ATT_DROPOUT,
                          output_hidden_states=True)

# The bare, pre-trained DistilBERT transformer model outputting raw hidden-states
# and without any specific head on top.
distilBERT = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

# Make DistilBERT layers untrainable
for layer in distilBERT.layers:
    layer.trainable = False

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
import focal_loss
from focal_loss import BinaryFocalLoss

In [ ]:
MAX_LENGTH = 128
LAYER_DROPOUT = 0.2
LEARNING_RATE = 5e-5
RANDOM_STATE = 42

def build_model(transformer, max_length=MAX_LENGTH):

    # Define weight initializer with a random seed to ensure reproducibility
    weight_initializer = tf.keras.initializers.GlorotNormal(seed=RANDOM_STATE)

    # Define input layers
    input_ids_layer = tf.keras.layers.Input(shape=(max_length,),
                                            name='input_ids',
                                            dtype='int32')
    input_attention_layer = tf.keras.layers.Input(shape=(max_length,),
                                                  name='input_attention',
                                                  dtype='int32')

    # DistilBERT outputs a tuple where the first element at index 0
    # represents the hidden-state at the output of the model's last layer.
    # It is a tf.Tensor of shape (batch_size, sequence_length, hidden_size=768).
    last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]

    # We only care about DistilBERT's output for the [CLS] token,
    # which is located at index 0 of every encoded sequence.
    # Splicing out the [CLS] tokens gives us 2D data.
    cls_token = last_hidden_state[:, 0, :]

    ##                                                 ##
    ## Define additional dropout and dense layers here ##
    ##                                                 ##

    # Define a single node that makes up the output layer (for binary classification)
    output = tf.keras.layers.Dense(1,
                                   activation='sigmoid',
                                   kernel_initializer=weight_initializer,
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(cls_token)

    # Define the model
    model = tf.keras.Model([input_ids_layer, input_attention_layer], output)

    # Compile the model
    model.compile(tf.keras.optimizers.Adam(lr=LEARNING_RATE),
                  loss=BinaryFocalLoss(gamma=2),
                  metrics=['accuracy'])

    return model

In [ ]:
y_train

7628      1
146225    0
11635     1
131853    0
146112    0
         ..
70153     1
133089    0
15417     1
125349    0
39333     1
Name: label, Length: 163074, dtype: int64

In [ ]:
X_train_ids

<tf.Tensor: shape=(163074, 128), dtype=int32, numpy=
array([[  101,  1999,  1996, ...,     0,     0,     0],
       [  101, 25416,  9863, ...,  2005,  1996,   102],
       [  101,  1996,  1000, ...,     0,     0,     0],
       ...,
       [  101,  6887,  6968, ...,     0,     0,     0],
       [  101,  1000,  4897, ...,     0,     0,     0],
       [  101,  1000,  1037, ...,  2393,  2115,   102]], dtype=int32)>

In [ ]:
EPOCHS = 6
BATCH_SIZE = 64
NUM_STEPS = len(X_train.index) // BATCH_SIZE

model = build_model(distilBERT)

# Train the model
train_history1 = model.fit(
    x = [X_train_ids, X_train_attention],
    y = y_train.to_numpy(),
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    steps_per_epoch = NUM_STEPS,

    verbose=2
)

Epoch 1/6
2548/2548 - 236s - loss: 0.0660 - accuracy: 0.8994 - 236s/epoch - 92ms/step
Epoch 2/6
2548/2548 - 228s - loss: 0.0598 - accuracy: 0.9103 - 228s/epoch - 90ms/step
Epoch 3/6
2548/2548 - 228s - loss: 0.0591 - accuracy: 0.9124 - 228s/epoch - 90ms/step
Epoch 4/6
2548/2548 - 229s - loss: 0.0587 - accuracy: 0.9125 - 229s/epoch - 90ms/step
Epoch 5/6
2548/2548 - 229s - loss: 0.0588 - accuracy: 0.9132 - 229s/epoch - 90ms/step
Epoch 6/6
2548/2548 - 229s - loss: 0.0583 - accuracy: 0.9125 - 229s/epoch - 90ms/step


In [ ]:
model.save('distilbert_unfined.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
new_model = tf.keras.models.load_model('distilbert_unfined.h5', custom_objects={"TFDistilBertModel": transformers.TFDistilBertModel})


In [ ]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 input_attention (InputLaye  [(None, 128)]                0         []                            
 r)                                                                                               
                                                                                                  
 tf_distil_bert_model_1 (TF  TFBaseModelOutput(last_hid   6636288   ['input_ids[0][0]',           
 DistilBertModel)            den_state=(None, 128, 768)   0          'input_attention[0][0]']     
                             , hidden_states=((None, 12                                       

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(x = [X_train_ids, X_train_attention], y = y_train.to_numpy(), verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

5097/5097 - 242s - loss: 0.0507 - accuracy: 0.9258 - 242s/epoch - 47ms/step
Restored model, accuracy: 92.58%


In [ ]:
X_test_ids

<tf.Tensor: shape=(54359, 128), dtype=int32, numpy=
array([[ 101, 2357, 2046, ...,    0,    0,    0],
       [ 101, 2000, 6869, ..., 1016, 1012,  102],
       [ 101, 1000, 1000, ...,    0,    0,    0],
       ...,
       [ 101, 4931, 1012, ...,    0,    0,    0],
       [ 101, 2023, 3931, ...,    0,    0,    0],
       [ 101, 2130, 2077, ...,    0,    0,    0]], dtype=int32)>

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(x = [X_test_ids, X_test_attention], y = y_test.to_numpy(), verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))



1699/1699 - 82s - loss: 0.0518 - accuracy: 0.9240 - 82s/epoch - 48ms/step
Restored model, accuracy: 92.40%


In [ ]:
inp_id, inp_att = tokenizer("you are a stupid")
new_model.predict(x = [X_test_ids, X_test_attention])


1699/1699 [==============================] - 81s 47ms/step


array([[0.9369638 ],
       [0.14883122],
       [0.0950274 ],
       ...,
       [0.15876141],
       [0.10111476],
       [0.83922166]], dtype=float32)

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

In [ ]:
new_model.metrics

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
target_names = ['SFW', 'NSFW']
print(classification_report(y_train, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         SFW       0.46      1.00      0.63     74914
        NSFW       0.00      0.00      0.00     88160

    accuracy                           0.46    163074
   macro avg       0.23      0.50      0.31    163074
weighted avg       0.21      0.46      0.29    163074



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# preds = new_model.predict([X_train_ids, X_train_attention])
pred_labels = np.argmax(preds.predictions, axis=1)
conf_matrix = metrics.confusion_matrix(y_train,pred_labels)
print('conf_matrix ',conf_matrix)

fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix(without preprocessing)', fontsize=18)
plt.show()

AttributeError: ignored

In [ ]:
import torch
import torch.nn.functional as F
from sklearn import metrics

y_preds = []
y_trues = []
for index,val_text in enumerate(val_texts):
     tokenized_val_text = tokenizer([val_text],
                                    truncation=True,
                                    padding=True,
                                    return_tensor='pt')
     logits = model(tokenized_val_text)
     prediction = F.softmax(logits, dim=1)
     y_pred = torch.argmax(prediction).numpy()
     y_true = val_labels[index]
     y_preds.append(y_pred)
     y_trues.append(y_true)